# Ejercicio de Bayes Ingenuo, Spam Filter
### Juan Pablo Fonseca Correa  138263

In [1]:
# importar librerías y leer la base de datos
import pandas as pd
import numpy as np
import math
from sys import maxint
from __future__ import division
from scipy.stats import norm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
df = pd.read_csv('spambase.data', sep=",",header = None) # leo la bd 

In [2]:
numRows = df.shape[0] # num filas totales
numCols = df.shape[1] # num columnas totales

In [3]:
# separar en conjuntos de entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(df[df.columns[0:-1]],df[df.columns[-1]], train_size=0.75)

c:\users\juanpa~1\desktop\env1\python~1\lib\site-packages\sklearn\model_selection\_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [4]:
numColsTrain = X_train.shape[0] # num filas para el train
numColsTest = X_test.shape[0] # num filas para el test
# aqui no voy a hacer conjunto de validacion, solo hice train y test

In [5]:
# calcular P(Spam) y P(no Spam)
probaSpam = len(Y_train.nonzero()[0])/len(Y_train)
probaNoSpam = 1 - probaSpam

# dentro del train, calcular promedios y desv estandar, dado que es o no Spam

# UNA FORMA DE HACERLO (separando spam y no spam, y luego con un for)
# dataTrainSpam = X_train[Y_train == 1]
# dataTrainNoSpam = X_train[Y_train == 0]
# promSpam = []
# desvestSpam = []
# promNoSpam = []
# desvestNoSpam = []
# for i in range(0,numCols-1):
#     promSpam.append(dataTrainSpam.iloc[:,i].mean())
#     desvestSpam.append(dataTrainSpam.iloc[:,i].std())
#     promNoSpam.append(dataTrainNoSpam.iloc[:,i].mean())
#     desvestNoSpam.append(dataTrainNoSpam.iloc[:,i].std())
     # ya tienes todos los promedios y desviaciones estandar para calcular todas las probas de clase (dist. Normal)

# OTRA FORMA MEJOR (todo junto):
promSpam = np.mean(X_train[Y_train == 1], axis = 0) # axis = 0 es para promediar columnas (verticalmente pues)
desvestSpam = np.std(X_train[Y_train == 1], axis = 0)
promNoSpam = np.mean(X_train[Y_train == 0], axis = 0)
desvestNoSpam = np.std(X_train[Y_train == 0], axis = 0)

In [6]:
# ahora vamos a probar con el test
logProbaSpam = math.log(probaSpam)
logProbaNoSpam = math.log(probaNoSpam)
clasifModelo = []
clasifOriginal = Y_test
normalesS = []
normalesNS = []
for j in range(0,numCols-1): # en este for guardo las normales que voy a utilizar
    normalesS.append(norm(promSpam[j],desvestSpam[j]))
    normalesNS.append(norm(promNoSpam[j],desvestNoSpam[j]))
for i in range(0,len(Y_test)):
    logProbaXDadoSpam = [] # log(P(Xi=xi|Spam)), para cada una de las Xi
    logProbaXDadoNoSpam = [] # log(P(Xi=xi|no Spam)), para cada una de las Xi
    for j in range(0,numCols-1):
        if(desvestSpam[j]>0): # solo agregas log(P(Xi=xi|Spam)) si la desvest es mayor a 0 (aporta informacion)
            normalDadoSpam=normalesS[j].pdf(X_test.iloc[i,j])
            if(normalDadoSpam>0):
                logProbaXDadoSpam.append(math.log(normalDadoSpam))
            elif(normalDadoSpam==0): # para probabilidades que nos dan cero
                logProbaXDadoSpam.append(-maxint - 1)
            else:
                print "Sale una proba negativa! hay un error..."
        if(desvestNoSpam[j]>0):
            normalDadoNoSpam=normalesNS[j].pdf(X_test.iloc[i,j])
            if(normalDadoNoSpam>0):
                logProbaXDadoNoSpam.append(math.log(normalDadoNoSpam))
            elif(normalDadoNoSpam==0):
                logProbaXDadoNoSpam.append(-maxint - 1)
            else:
                print "Sale una proba negativa! hay un error..."
    valorSpam = logProbaSpam + np.sum(logProbaXDadoSpam)
    valorNoSpam = logProbaNoSpam + np.sum(logProbaXDadoNoSpam)
    if(valorSpam >= valorNoSpam):
        clasifModelo.append(1)
    else:
        clasifModelo.append(0)

# no sumas probabilidades = 0, y no sumas desvest = 0 ( es la solución del profe ) 

In [14]:
# evaluación (aquí con precision y recall)
truePositives = 0
selected = 0
relevant = 0

for i in range(0,len(Y_test)):
    if(clasifModelo[i] == 1 and clasifOriginal.values[i] == 1):
        truePositives += 1
    if(clasifModelo[i] == 1):
        selected += 1
    if(clasifOriginal.values[i] == 1):
        relevant += 1
precision = truePositives / selected
recall = truePositives / relevant
print 'Precision: {0}'.format(precision)
print 'Recall: {0}'.format(recall)

Precision: 0.724615384615
Recall: 0.967145790554


In [15]:
# evaluación con matriz de confusión
mc = confusion_matrix(clasifOriginal,clasifModelo)
print "Matriz de confusión:"
print "{0}".format(mc)
print "Verdaderos negativos: {0}".format(mc[0][0])
print "Falsos positivos: {0}".format(mc[0][1])
print "Falsos negativos: {0}".format(mc[1][0])
print "Verdaderos positivos: {0}".format(mc[1][1])

Matriz de confusión:
[[485 179]
 [ 16 471]]
Verdaderos negativos: 485
Falsos positivos: 179
Falsos negativos: 16
Verdaderos positivos: 471
